In [1]:
import numpy as np
import keras

Using TensorFlow backend.


In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D,Activation

In [3]:
from game2048.game import Game
from game2048.displays import Display, IPythonDisplay
from game2048.agents import Agent, RandomAgent, ExpectiMaxAgent

In [29]:
display1 = Display()
display2 = IPythonDisplay()

In [ ]:
[[] for _ in range(10)]

In [30]:
game = Game(4, random=False)
display1.display(game)
display2.display(game)
agent = RandomAgent(game, display=display1)
agent.play(verbose=True)

State:	       0       0       0       0
	       0       0       4       2
	       0       0       0       0
	       0       0       0       0
Score: 4


,,,
,,4,2
,,,
,,,


Iter: 1
======Direction: right======
State:	       0       0       0       0
	       0       0       4       2
	       0       0       0       0
	       0       0       0       4
Score: 4
Iter: 2
======Direction: up======
State:	       0       0       4       2
	       0       0       0       4
	       0       2       0       0
	       0       0       0       0
Score: 4
Iter: 3
======Direction: left======
State:	       4       2       0       0
	       4       0       0       0
	       2       0       0       0
	       0       0       2       0
Score: 4
Iter: 4
======Direction: left======
State:	       4       2       0       0
	       4       2       0       0
	       2       0       0       0
	       2       0       0       0
Score: 4
Iter: 5
======Direction: down======
State:	       0       0       0       0
	       0       0       2       0
	       8       0       0       0
	       4       4       0       0
Score: 8
Iter: 6
======Direction: up======
State:	       8       4       2 

In [ ]:
game.board

In [ ]:
agent.step()

In [31]:
#every movement
y=np.array(agent.y)
y.shape

(26,)

In [13]:
#corresponding game board
agent.x

array([[[  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   4.,   0.],
        [  0.,   0.,   4.,   0.]],

       [[  0.,   0.,   0.,   2.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   8.,   0.]],

       [[  0.,   0.,   0.,   2.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  2.,   0.,   0.,   8.]],

       [[  2.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  2.,   8.,   2.,   0.]],

       [[  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  4.,   8.,   2.,   2.]],

       [[  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   4.],
        [  0.,   4.,   8.,   4.]],

       [[  0.,   4.,   8.,   8.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   0.],
        [  0.,   0.,   0.,   4.]],

       [[  4.,  16.,   0.,   0.],


In [ ]:
game = Game(4, random=False)
agent = Agent(game, display=display2)
agent.play(verbose=True)

In [47]:
arrxx=np.empty((0,4,4))
arryy=np.empty((0,))
for i in range(1,1):
    game=Game(4,score_to_win=2048,random=False)
    agent=ExpectiMaxAgent(game,display=None)
    agent.play(verbose=False)
    arrxx=np.concatenate((arrxx,agent.x),axis=0)
    y=np.array(agent.y)
    arryy=np.concatenate((arryy,y),axis=0)

In [49]:
arrxx.shape

(27994, 4, 4)

In [50]:
arryy.shape

(27994,)

In [51]:
np.save('arrxx_test',arrxx)
np.save('arryy_test',arryy)

In [4]:
arrxx_test=np.load('arrxx_test.npy')
arryy_test=np.load('arryy_test.npy')
arrxx_train=np.load('arrxx.npy')
arryy_train=np.load('arryy.npy')

In [15]:
arrxx_test.shape

(27994, 4, 4)

In [18]:
#input a 3-D array
def grid_ohe(arr):
    
    CAND=16
    map_table={2**i:i for i in range(1,CAND)}
    map_table[0]=0
    
    ret=np.zeros(shape=arr.shape+(CAND,),dtype=bool)
    for a in range(arr.shape[0]):
        for b in range(arr.shape[1]):
            for c in range(arr.shape[2]):
                ret[a,b,c,map_table[arr[a,b,c]]]=1
    return ret            

In [19]:
arrxx_test_ohe=grid_ohe(arrxx_test)
arrxx_test_ohe.shape

(27994, 4, 4, 16)

In [8]:
arrxx_train_ohe=grid_ohe(arrxx_train)
arrxx_train_ohe.shape

(107494, 4, 4, 16)

In [9]:
arrxx_ohe[1,1,1,1]

False

In [17]:
arryy.shape

(27994,)

In [9]:
arryy_test_ohe=keras.utils.to_categorical(arryy_test)
arryy_test_ohe.shape

(27994, 4)

In [10]:
arryy_train_ohe=keras.utils.to_categorical(arryy_train)
arryy_train_ohe.shape

(107494, 4)

In [27]:
# define the RNN model
model = Sequential()

model.add(Conv2D(128, (1, 4),padding='SAME', activation='relu', input_shape=(4, 4, 16),kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (4, 1),padding='SAME', activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (2, 2),padding='SAME', activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3),padding='SAME', activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Conv2D(128, (4, 4),padding='SAME', activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))

model.summary()
# define the object function, optimizer and metrics
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 4, 4, 128)         8320      
_________________________________________________________________
dropout_7 (Dropout)          (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 4, 4, 128)         65664     
_________________________________________________________________
dropout_8 (Dropout)          (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 128)         65664     
_________________________________________________________________
dropout_9 (Dropout)          (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 4, 128)         147584    
__________

In [12]:
# train
model.fit(arrxx_train_ohe, arryy_train_ohe,
                    batch_size = 128, 
                    epochs = 15,
                    verbose =1,
                    validation_data = (arrxx_test_ohe, arryy_test_ohe))

Train on 107494 samples, validate on 27994 samples
Epoch 1/15
107494/107494 [==============================] - 154s 1ms/step - loss: 1.1606 - acc: 0.3914 - val_loss: 1.1108 - val_acc: 0.4176
Epoch 2/15
107494/107494 [==============================] - 154s 1ms/step - loss: 1.1055 - acc: 0.4235 - val_loss: 1.0868 - val_acc: 0.4378
Epoch 3/15
107494/107494 [==============================] - 153s 1ms/step - loss: 1.0792 - acc: 0.4513 - val_loss: 1.0582 - val_acc: 0.4673
Epoch 4/15
107494/107494 [==============================] - 168s 2ms/step - loss: 1.0510 - acc: 0.4737 - val_loss: 1.0248 - val_acc: 0.4932
Epoch 5/15
107494/107494 [==============================] - 180s 2ms/step - loss: 1.0205 - acc: 0.4963 - val_loss: 0.9920 - val_acc: 0.5170
Epoch 6/15
107494/107494 [==============================] - 296s 3ms/step - loss: 0.9937 - acc: 0.5144 - val_loss: 0.9709 - val_acc: 0.5289
Epoch 7/15
107494/107494 [==============================] - 197s 2ms/step - loss: 0.9689 - acc: 0.5291 - val_

In [13]:
score_train =model.evaluate(arrxx_train_ohe, arryy_train_ohe, verbose=0)
score_test =model.evaluate(arrxx_test_ohe, arryy_test_ohe, verbose=0)
print('Training loss: %.4f, Training accuracy: %.4f' % (score_train[0],score_train[1]))
print('Testing loss: %.4f, Testing accuracy: %.4f' % (score_test[0],score_test[1]))

Training loss: 0.7424, Training accuracy: 0.6494
Testing loss: 0.8604, Testing accuracy: 0.5977


In [26]:
#save model
from keras.models import load_model
model.save('my_model.h5')

In [75]:
del model  # deletes the existing model

In [77]:
model = load_model('my_model.h5')

In [22]:
x=np.expand_dims(arrxx_test_ohe[0],axis=0)
x.shape

(1, 4, 4, 16)

In [24]:
model.predict_classes(x)[0]

1

In [ ]:
class MyAgent(Agent):

    def step(self):
        '''To define the agent's 1-step behavior given the `game`.
        You can find more instance in [`agents.py`](game2048/agents.py).
        
        :return direction: 0: left, 1: down, 2: right, 3: up
        '''
        direction = some_function(self.game)
        return direction

In [18]:
%%time
game = Game(4, score_to_win=512, random=False)
display2.display(game)
agent = ExpectiMaxAgent(game, display=None)
agent.play(verbose=False)

,4,,
,,,
,,,
,,4,


CPU times: user 5.52 s, sys: 94.4 ms, total: 5.61 s
Wall time: 9.98 s


In [19]:
#every movement
y=np.array(agent.y)
y

array([0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 2, 2, 1, 3, 2, 3, 1, 2, 2, 2, 1, 2,
       1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1,
       0, 1, 2, 0, 2, 1, 2, 2, 1, 2, 2, 3, 1, 2, 1, 1, 2, 0, 0, 1, 2, 2, 2,
       1, 0, 0, 1, 0, 0, 0, 1, 2, 2, 1, 1, 2, 1, 0, 1, 1, 1, 0, 0, 0, 2, 2,
       1, 1, 0, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, 3, 3, 1, 3, 0, 2, 2, 1, 2, 2,
       1, 1, 2, 2, 3, 2, 1, 2, 3, 1, 3, 2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 1, 1,
       1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1,
       1, 2, 2, 2, 2, 1, 3, 1, 2, 3, 2, 1, 2, 3, 3, 3, 1, 2, 3, 2, 3, 1, 2,
       0, 1, 1, 0, 1, 1, 2])

In [20]:
#corresponding game board
agent.x

array([[[   0.,    4.,    0.,    0.],
        [   0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.],
        [   0.,    0.,    4.,    0.]],

       [[   4.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.],
        [   4.,    0.,    0.,    2.]],

       [[   0.,    0.,    0.,    4.],
        [   0.,    0.,    0.,    0.],
        [   0.,    0.,    0.,    0.],
        [   2.,    0.,    4.,    2.]],

       ..., 
       [[  16.,    2.,    0.,    0.],
        [   4.,    2.,    4.,    2.],
        [   2.,    8.,  128.,    0.],
        [   4.,    2.,  128.,  256.]],

       [[  16.,    0.,    0.,    2.],
        [   4.,    4.,    0.,    0.],
        [   2.,    8.,    4.,    2.],
        [   4.,    2.,  256.,  256.]],

       [[  16.,    0.,    2.,    0.],
        [   4.,    4.,    0.,    0.],
        [   2.,    8.,    4.,    4.],
        [   4.,    2.,  256.,  256.]]])

In [ ]:
%%time
# a random start
game = Game(4, score_to_win=2048, random=True)
display2.display(game)
agent = ExpectiMaxAgent(game, display=display2)
agent.play(verbose=True)

In [ ]:
for _ in range(10):
    print("The ExpectiMax agent always search a fixed solution given certain board:",
          agent.step())

In [ ]:
print("Running the loop manually...")

game = Game(4, random=False, enable_rewrite_board=False)
agent = RandomAgent(game)

for _ in range(10):
    direction = agent.step()
    print("Moving to direction `%s`..."%direction)
    game.move(direction)
    display1.display(game)
    display2.display(game)

In [ ]:
game.board = game.board

In [ ]:
game.enable_rewrite_board = True

In [79]:
game.board = game.board

NameError: name 'game' is not defined

In [80]:
game.board

NameError: name 'game' is not defined

In [20]:
display1 = Display()
display2 = IPythonDisplay()

game = Game(4, random=False)
agent = MyAgent(game, display=display2)
agent.play(verbose=True)

NameError: name 'MyAgent' is not defined